In [ ]:
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# loading the dataset 

In [ ]:
df=pd.read_csv('EDA_winequality.csv',sep=',')
df
df.head(10)

# EDA

In [ ]:
df.shape


In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# summary statistics

In [ ]:
df.describe(
)

key observations here
1.   mean is more than median in all columns
2. there is a huge diffence in 75% and max values .. 
these two obseravtion suggests there are outliers inn the dataset

exploring data variables 

In [ ]:
df.quality.unique()

In [ ]:
# it is showing that quality has most values concetrated in the categoris of 5,6,7
df.quality.value_counts()

In [ ]:
# now visualizing the data

In [ ]:
# check missing variables 
df.isnull()

In [ ]:
df.isnull().sum()

Data Visualization 

In [ ]:
# here data  set has no mising values 
sns.heatmap(df.isnull())

checking the correlation

In [ ]:
dfcor=df.corr()

In [ ]:
dfcor

In [ ]:
sns.heatmap(dfcor,annot=True)

In [ ]:
sns.color_palette?

In [ ]:
sns.heatmap?

In [ ]:
import matplotlib.pyplot as plt

here , heatmap shows the dark colors are highly corelated

In [ ]:
# annot true shows value in heatmap
plt.Figure(figsize=(6,4))
sns.heatmap(dfcor,cmap='Blues',annot=True)

light shades are highly corelated

quality is highly corelated with the alcolhol

alcohol is negitively corelated with density with -0.5

volatile acidity is negatively corelated with quality

hence we can drop volatile acidity from the dataset if we aer doing the linear regression

In [ ]:
plt.Figure(figsize=(40,15))
sns.heatmap(dfcor,cmap='YlOrRd_r',annot=True)

In [ ]:
df.columns

In [ ]:
#univariate analysis

In [ ]:
df['fixed acidity'].plot.box()

In [ ]:
df['alcohol'].plot.box()

In [ ]:
df['free sulfur dioxide'].plot.box()

In [ ]:
df.shape

In [ ]:
plt.subplot?

In [ ]:
collist=df.columns.values
ncol=12
nrows=10

In [ ]:
plt.figure(figsize=(5*ncol,5*ncol))
for i in range(1,len(collist)):
    plt.subplot(nrows,ncol,i+1)
    sns.boxplot(df[collist[i]],color='green',orient='v')
    plt.tight_layout()

nnow to check the distribution of skewness

In [ ]:
sns.distplot(df['density'])

In [ ]:
sns.distplot(df['citric acid'])

In [ ]:
plt.figure(figsize=(16,16))
for i in range(0,len(collist)):
    plt.subplot(nrows,ncol,i+1)
    sns.distplot(df[collist[i]])

here only ph and density are normal distributed 


other are rightly skewed / positively skewed

In [ ]:
# bivariate analysis


In [ ]:
plt.scatter(df['pH'],df['quality'])

In [ ]:
plt.scatter(df['volatile acidity'],df['quality'])
plt.show()

In [ ]:
sns.pairplot(df)

In [ ]:
# remove the missng values
#drop the negatively coorelated cols
# remove the outliers

In [ ]:
# Data cleaning is done

In [ ]:
df.drop('volatile acidity',axis=1,inplace=True)
df.head()

Removing the outliers

In [ ]:
from scipy.stats import zscore
import numpy as np

In [ ]:
z=np.abs(zscore(df))
z

In [ ]:
threshold=3
print(np.where(z>3))

In [ ]:
z[15][5]

In [ ]:
#outliers
z[1589][2]

In [ ]:
df_new=df[(z<3).all(axis=1)]

In [ ]:
df_new

In [ ]:
df_new.shape

In [ ]:
df_new.skew()

In [ ]:
x=df_new.iloc[:,0:-1]
x.shape
x

In [ ]:
# removing skewness by transforming data
from sklearn.preprocessing import power_transform

In [ ]:
x=power_transform(x,method='yeo-johnson')

In [ ]:
y=df_new.iloc[:,-1]
y.shape

#modelling the data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x=sc.fit_transform(x)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.22,random_state=0)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dtc=DecisionTreeClassifier()
dtc.fit(x_train,y_train)
dtc.score(x_train,y_train)
preddtc=dtc.predict(x_test)
print("accuracy score",accuracy_score(y_test,preddtc))
print("confusion matrix\n",confusion_matrix(y_test,preddtc))
print("classification report\n",classification_report(y_test,preddtc))

In [ ]:
rfc=RandomForestClassifier()
rfc.fit(x_train,y_train)
rfc.score(x_train,y_train)
predrfc=rfc.predict(x_test)
print("accuracy score",accuracy_score(y_test,predrfc))
print("confusion matrix\n",confusion_matrix(y_test,predrfc))
print("classification report\n",classification_report(y_test,predrfc))

In [ ]:
gbc=GradientBoostingClassifier()
gbc.fit(x_train,y_train)
gbc.score(x_train,y_train)
predgbc=gbc.predict(x_test)
print("accuracy score",accuracy_score(y_test,predgbc))
print("confusion matrix\n",confusion_matrix(y_test,predgbc))
print("classification report\n",classification_report(y_test,predgbc))

In [ ]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)
knn.score(x_train,y_train)
predknn=knn.predict(x_test)
print("accuracy score",accuracy_score(y_test,predknn))
print("confusion matrix\n",confusion_matrix(y_test,predknn))
print("classification report\n",classification_report(y_test,predknn))

#Conclusion- RandomForestClassifier is giving the highest accuracy score with .68

In [ ]:
a=np.array(y_test)
a

In [ ]:
df_con=pd.DataFrame({'original':a,"predicted":predrfc},index=range(len(a)))
df_con

In [ ]:
# saving the prediction model

import pickle
filename="WineQuality.pkl"
pickle.dump(predrfc,open(filename,'wb'))